In [19]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [20]:
dict_tablas = pickle.load(open('dict_tablas','rb'))
df_historical_data = pd.read_csv(r'Datasets\Clean_Fifa_WorldCup_Matches.csv')
df_fixture = pd.read_csv(r'Datasets\Fifa_WorldCup_Fixture.csv')

In [23]:
dict_tablas['Grupo F']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Belgium,0,0,0,0,0,0,0,0
1,2,Canada,0,0,0,0,0,0,0,0
2,3,Morocco,0,0,0,0,0,0,0,0
3,4,Croatia,0,0,0,0,0,0,0,0


## 1.- Calcular Nivel de equipo

In [25]:
#Dividir df en df_home y df_away
df_historical_data


,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
895,Russia,Croatia,2018,2,2
896,France,Belgium,2018,1,0
897,Croatia,England,2018,2,1
898,Belgium,England,2018,2,0


In [26]:
#Dividir df en df_home y df_away
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [27]:
#Renombrar Columnas
df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalScored','AwayGoals':'GoalConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalConceded','AwayGoals':'GoalScored'})

In [30]:
#Concatenar df_home y df_away, hacer grupo por team y calcular promedio
df_team_strenght = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
df_team_strenght

,GoalScored,GoalConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935
